In [1]:
import io
import pandas as pd
from connections.aws import AWS

In [2]:
""" INITIALIZE CONNECTION """
# create connection
aws_connection = AWS()
aws_connection.connect()

[AWS]: Port 5433 is free.
[AWS]: Connected to RDS endpoint.


$\textbf{Marker Processing}$

Loads, checks format, and rotates all TRC files for each pitcher.

In [84]:
import biomech
from biomech.processing.trc import *
from biomech.processing import rotate_data

In [4]:
""" S3 OBJECT LISTING """
# NOTE: have to update IAM user permissions to allow S3 access
s3_objects_pro = aws_connection.list_s3_objects(prefix='PRO/')

# get all unique subject IDs
subject_ids = set()
for obj in s3_objects_pro:
    subject_id = obj.split('/')[1]
    subject_ids.add(subject_id)

# convert to list
subject_ids = sorted(list(subject_ids))

In [5]:
# extract (raw, not yet usable) CSV & TRC objects
s3_objects_csv = [obj for obj in s3_objects_pro if obj.endswith('.csv') and obj.split('.csv')[0].split('/')[-1]  in ['trackman', 'rapsodo']]
s3_objects_trc = [obj for obj in s3_objects_pro if obj.endswith('.trc')]

# load subject info (for throwing hand)
subject_info_bytes = aws_connection.load_s3_object('subjects/summary/subject_info.csv', return_info=False)
subject_info = pd.read_csv(io.BytesIO(subject_info_bytes))

In [ ]:
# TODO: re-reun all trials through marker processing w/ TRC format check

In [89]:
""" TRC READING """

# constants
FRAME_RATE = 480
PARENT_DIR = 'subjects'

# store error trials for investigation
error_trials = []

# iterate through all TRCs
for trc_file in s3_objects_trc:
    
    # load subject info
    subject_id = trc_file.split('/')[1]
    throwing_hand = subject_info[subject_info['subject_id'] == int(subject_id)]['throws'].values[0]

    try:
        # load bytes from S3
        trc_bytes, trc_info = aws_connection.load_s3_object(trc_file)
        
        # read TRC data (body) & rotate to match OpenSim coordinates
        trc_body = parse_trc_body(trc_bytes)
        trc_body_checked, format_valid = check_trc_format(trc_body)                       # check for valid TRC format (Time, Frame#)
        trc_body_rotated = rotate_data(trc_body_checked)

        # filter markers to correct throwing hand
        if throwing_hand == 'right':
            trc_body_clean = trc_body_rotated[['Frame#', 'Time'] + biomech.processing.trc.__markers_right__]
        elif throwing_hand == 'left':
            trc_body_clean = trc_body_rotated[['Frame#', 'Time'] + biomech.processing.trc.__markers_left__]

        # get TRC header information
        trc_header = create_trc_header(trc_info['study_id'], trc_body_clean, throwing_hand, FRAME_RATE)

        # put object to S3
        aws_connection.upload_trc_to_s3(trc_header, trc_body_clean, f'{PARENT_DIR}/{trc_info["subject_id"]}/trc_raw/{trc_info["study_id"]}.trc')

    except Exception as e:
        print(f"Error processing TRC file {trc_file} for subject {subject_id}: {e}")
        
        # log error
        error_trials.append({
            'subject_id': subject_id,
            'trc_file': trc_file,
            'format_valid': format_valid,
            'error': str(e)
        })
        
        continue

[AWS]: TRC file written to s3://pitch-ml/subjects/2609/trc_raw/2609_01.trc
[AWS]: TRC file written to s3://pitch-ml/subjects/2609/trc_raw/2609_02.trc
[AWS]: TRC file written to s3://pitch-ml/subjects/2609/trc_raw/2609_03.trc
[AWS]: TRC file written to s3://pitch-ml/subjects/2609/trc_raw/2609_04.trc
[AWS]: TRC file written to s3://pitch-ml/subjects/2609/trc_raw/2609_05.trc
[AWS]: TRC file written to s3://pitch-ml/subjects/2609/trc_raw/2609_06.trc
[AWS]: TRC file written to s3://pitch-ml/subjects/2609/trc_raw/2609_07.trc
[AWS]: TRC file written to s3://pitch-ml/subjects/2609/trc_raw/2609_08.trc
[AWS]: TRC file written to s3://pitch-ml/subjects/2609/trc_raw/2609_09.trc
[AWS]: TRC file written to s3://pitch-ml/subjects/2609/trc_raw/2609_10.trc
[AWS]: TRC file written to s3://pitch-ml/subjects/2609/trc_raw/2609_11.trc
[AWS]: TRC file written to s3://pitch-ml/subjects/2609/trc_raw/2609_12.trc
[AWS]: TRC file written to s3://pitch-ml/subjects/2609/trc_raw/2609_13.trc
[AWS]: TRC file written t

KeyboardInterrupt: 

$\textit{Upload Processing Summary}$

Now includes `format_valid` for traceback debugging.

In [ ]:
# write error trials to file for later investigation
error_trials_summary = pd.DataFrame(error_trials)
aws_connection.upload_to_s3(
    error_trials_summary,
    s3_key=f'subjects/summary/error_trials_summary.csv'
)